In [17]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

from VQLSSVM import VQLSSVM

np.set_printoptions(precision=4, suppress=True)

In [18]:
shots: int = 10000
gamma: float = 0.01 # regularization parameter
subsetSize: int = 7
classToFilterOut: int = 2

In [19]:
def prepareDataset(normalizeValues: bool = False, dataset: str = "iris") -> (np.ndarray, np.ndarray, np.array, np.array):
    if dataset == "iris":
        X,y = datasets.load_iris(return_X_y=True)
        X = X[y!=classToFilterOut]
        y = y[y!=classToFilterOut]
    elif dataset == "breastCancer":
        X,y = datasets.load_breast_cancer(return_X_y=True)
    elif dataset == "dummyData":
        X, y = datasets.make_classification(
            n_samples=50, n_features=2, n_informative=2, n_redundant=0, random_state=45
        )
    else: 
        raise ValueError("Dataset not yet implemented")
    
    y: np.array = np.where(y == 0, -1, 1) # might cause some problems in a future

    if normalizeValues:
        max = np.max(X, axis=0)
        min = np.min(X, axis=0)
        X = (2*X - min - max) / (max - min)
    return train_test_split(X, y, test_size=(X.shape[0]-subsetSize)/(X.shape[0]), random_state=42)

In [20]:
xTrain, xTest, yTrain, yTest = prepareDataset(normalizeValues=True, dataset="breastCancer")
print(xTrain)

[[ 0.1055 -0.4988  0.0726 -0.2081 -0.0472 -0.4441 -0.3172 -0.1387 -0.0848
  -0.4874 -0.5645 -0.4604 -0.61   -0.6875 -0.5656 -0.7182 -0.8312 -0.3934
  -0.6463 -0.7461  0.0189 -0.3134 -0.0538 -0.3296  0.0451 -0.6084 -0.4773
   0.1505 -0.476  -0.6128]
 [-0.4227 -0.5942 -0.4217 -0.6806 -0.0093 -0.3398 -0.7859 -0.6909 -0.0838
  -0.2355 -0.9466 -0.8287 -0.941  -0.9706 -0.8379 -0.7487 -0.9142 -0.7541
  -0.7496 -0.8943 -0.5326 -0.5485 -0.545  -0.7811 -0.2072 -0.5143 -0.6981
  -0.4995 -0.3617 -0.6486]
 [-0.8195 -0.6666 -0.7927 -0.9147 -0.1839 -0.1797 -0.5967 -0.7145 -0.1495
   0.6782 -0.6997 -0.7825 -0.7728 -0.9304  0.0536  0.3733 -0.7136 -0.3309
  -0.5067  0.4534 -0.8717 -0.8054 -0.879  -0.9512 -0.3458 -0.5803 -0.7709
  -0.6711 -0.7284 -0.3   ]
 [-0.559  -0.417  -0.5663 -0.7718  0.1117 -0.495  -0.6687 -0.6536 -0.2515
  -0.358  -0.8591 -0.4268 -0.8683 -0.9484 -0.5358 -0.6868 -0.8513 -0.4704
  -0.7814 -0.7965 -0.6293 -0.081  -0.6504 -0.8346  0.2894 -0.5368 -0.5411
  -0.1629 -0.5107 -0.5287]
 [-0

In [23]:
vqlssvmVectors: VQLSSVM = VQLSSVM(gamma, shots)

# methods = ['COBYLA', 'SLSQP',
# slsqp does not stop
           
# methods = ['COBYLA', 'SLSQP','BFGS', 'Powell',
# [ 'Nelder-Mead', 'L-BFGS-B', 'TNC', 'trust-constr', 'dogleg',
methods =  ['trust-krylov']
# methods = ['COBYLA', 'SLSQP']
costs = []
for method in methods:
    print("Method: " + method)
    vqlssvmVectors.train(xTrain, yTrain, quantumSimulation=False, verbose=True, iterations = 200, method=method)

    vqlssvmVectors.plotCost()
    cost = vqlssvmVectors.getCostHistory()
    print("Cost history:")
    print(cost)
    costs.append(cost)

Method: trust-krylov
Condition number of the matrix:  4397.40886173452
LS-SVM Matrix:
 [[  0.       1.       1.       1.       1.       1.       1.       1.    ]
 [  1.     106.3387   7.9382   4.45     7.0972   9.6707   7.1253   8.6627]
 [  1.       7.9382 112.9724   9.9     11.3853  14.9474  10.5049  13.5895]
 [  1.       4.45     9.9    112.9916   8.8691  10.9625   7.0351   9.8842]
 [  1.       7.0972  11.3853   8.8691 111.003   12.9767   9.92    12.5543]
 [  1.       9.6707  14.9474  10.9625  12.9767 119.5408  11.9793  17.2147]
 [  1.       7.1253  10.5049   7.0351   9.92    11.9793 109.8428  11.4212]
 [  1.       8.6627  13.5895   9.8842  12.5543  17.2147  11.4212 116.0885]]
['III', 'IIX', 'IIZ', 'IXI', 'IXX', 'IXZ', 'IYY', 'IZI', 'IZX', 'IZZ',
 'XII', 'XIX', 'XIZ', 'XXI', 'XXX', 'XXZ', 'XYY', 'XZI', 'XZX', 'XZZ',
 'YIY', 'YXY', 'YYI', 'YYX', 'YYZ', 'YZY', 'ZII', 'ZIX', 'ZIZ', 'ZXI',
 'ZXX', 'ZXZ', 'ZYY', 'ZZI', 'ZZX', 'ZZZ']
Pauli matrix coeffitients [(98.59722459143913+0j), (8.82

ValueError: ('Jacobian is required for trust region ', 'exact minimization.')

In [ ]:
# methods = ['COBYLA', 'SLSQP','BFGS', 'Powell']#, 'Nelder-Mead', 'L-BFGS-B', 'TNC', 'trust-constr', 'dogleg', 'trust-ncg', 'trust-exact', 'trust-krylov']
# methods = ['Nelder-Mead', 'L-BFGS-B', 'TNC', 'trust-constr']
import csv
# where one column is iteration number and the other is cost

with open('costOptimizers2.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Iteration'] + methods)
    for i in range(200):
        values = [i]
        for j in range(len(costs)):
            values.append(costs[j][i])
        writer.writerow(values)

In [28]:
import pandas as pd

# Load the first CSV file
df1 = pd.read_csv('costOptimizers1.csv')

# Load the second CSV file and drop its first column
df2 = pd.read_csv('costOptimizers2.csv')
df2 = df2.iloc[:, 1:]  # Remove the first column

# Combine the two DataFrames
combined_df = pd.concat([df1, df2], ignore_index=False, axis=1)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('costOptimizers.csv', index=False)